In [ ]:
### <span style="color:#CA762B">**Advanced Exception Handling in Python**</span>

This notebook covers advanced exception handling techniques, custom exceptions, and best practices for error management in Python.


### <span style="color:#CA762B">**Custom Exceptions**</span>

Creating custom exceptions for better error handling and code organization.


In [ ]:
# Basic custom exception
class ValidationError(Exception):
    """Raised when data validation fails."""
    pass

# Custom exception with additional information
class DatabaseError(Exception):
    def __init__(self, message, error_code):
        self.message = message
        self.error_code = error_code
        super().__init__(self.message)

# Using custom exceptions
def validate_age(age):
    if not isinstance(age, int):
        raise ValidationError("Age must be an integer")
    if age < 0:
        raise ValidationError("Age cannot be negative")
    return age

try:
    validate_age(-5)
except ValidationError as e:
    print(f"Validation failed: {e}")


### <span style="color:#CA762B">**Exception Hierarchies**</span>

Building hierarchical exception structures for better error organization.


In [ ]:
class AppError(Exception):
    """Base exception for the application."""
    pass

class ConfigError(AppError):
    """Configuration-related errors."""
    pass

class NetworkError(AppError):
    """Network-related errors."""
    pass

class DatabaseError(AppError):
    """Database-related errors."""
    pass

def simulate_error(error_type):
    if error_type == "config":
        raise ConfigError("Invalid configuration")
    elif error_type == "network":
        raise NetworkError("Connection failed")
    elif error_type == "database":
        raise DatabaseError("Query failed")

# Handling different types of errors
for error_type in ["config", "network", "database"]:
    try:
        simulate_error(error_type)
    except AppError as e:
        print(f"{type(e).__name__}: {str(e)}")


### <span style="color:#CA762B">**Context Managers for Error Handling**</span>

Using context managers to handle resources and errors gracefully.


In [ ]:
from contextlib import contextmanager
import sys
from typing import Optional

@contextmanager
def error_handler(error_types: tuple = (Exception,), 
                 default_value: Optional[any] = None,
                 logger=None):
    try:
        yield
    except error_types as e:
        if logger:
            logger.error(f"Error occurred: {e}")
        if default_value is not None:
            return default_value
        raise

# Example usage
with error_handler((ValueError, TypeError), default_value=0):
    result = int("not a number")
    print(result)
print("Execution continues...")


### <span style="color:#CA762B">**Advanced Try/Except Patterns**</span>

Complex exception handling patterns and techniques.


In [ ]:
# Multiple exception handlers
def process_data(data):
    try:
        # Attempt to process data
        value = int(data)
        result = 100 / value
        return result
    except ValueError as e:
        print(f"Invalid data format: {e}")
        return None
    except ZeroDivisionError as e:
        print(f"Division by zero: {e}")
        return float('inf')
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise  # Re-raise unexpected exceptions
    else:
        print("Processing completed successfully")
    finally:
        print("Cleanup operations")

# Test different scenarios
test_values = ["10", "0", "not a number", "5"]
for value in test_values:
    print(f"\nProcessing {value}:")
    process_data(value)


### <span style="color:#CA762B">**Exception Chaining**</span>

Understanding and using exception chaining for better error tracking.


In [ ]:
class DataProcessingError(Exception):
    pass

def process_file(filename):
    try:
        with open(filename, 'r') as f:
            data = f.read()
            return int(data)
    except FileNotFoundError as e:
        raise DataProcessingError("Could not find the data file") from e
    except ValueError as e:
        raise DataProcessingError("File contains invalid data") from e

# Example usage
try:
    process_file("nonexistent.txt")
except DataProcessingError as e:
    print(f"Error: {e}")
    if e.__cause__:
        print(f"Caused by: {e.__cause__}")


### <span style="color:#CA762B">**Best Practices**</span>

Guidelines and patterns for effective exception handling.


In [ ]:
# 1. Always catch specific exceptions
def example_specific_catch(value):
    try:
        return int(value)
    except ValueError:  # Specific exception
        return 0

# 2. Use exception chaining with raise from
def example_chaining(data):
    try:
        process_data(data)
    except ValueError as e:
        raise RuntimeError("Processing failed") from e

# 3. Clean up resources with context managers
class Resource:
    def __init__(self, name):
        self.name = name
        print(f"Acquiring {name}")
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Releasing {self.name}")
        return False  # Don't suppress exceptions

# Example usage
with Resource("database"):
    # Resource is automatically released even if an exception occurs
    raise ValueError("Something went wrong")


### <span style="color:#CA762B">**Practical Applications**</span>

Real-world examples of exception handling patterns.


In [ ]:
class DatabaseConnection:
    def __init__(self, host):
        self.host = host
        self.connected = False
    
    def connect(self):
        print(f"Connected to {self.host}")
        self.connected = True
    
    def disconnect(self):
        if self.connected:
            print(f"Disconnected from {self.host}")
            self.connected = False
    
    def query(self, sql):
        if not self.connected:
            raise ConnectionError("Not connected to database")
        if "error" in sql.lower():
            raise DatabaseError("Query failed", 500)
        return f"Result of: {sql}"

@contextmanager
def database_session(host):
    db = DatabaseConnection(host)
    try:
        db.connect()
        yield db
    except DatabaseError as e:
        print(f"Database error {e.error_code}: {e.message}")
    finally:
        db.disconnect()

# Example usage
with database_session("localhost") as db:
    try:
        result = db.query("SELECT * FROM users")
        print(result)
        result = db.query("error in query")  # This will raise an error
    except ConnectionError as e:
        print(f"Connection error: {e}")